In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import string
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
from sklearn.svm import LinearSVC

In [2]:
data = pd.read_csv('amazon_data.txt', sep='\t', header=None)

In [3]:
data.columns = ["Review", "Sentiment"]
data.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
data["Sentiment"].value_counts() # checking data

1    500
0    500
Name: Sentiment, dtype: int64

In [5]:
data.isnull().sum() # checking for null values

Review       0
Sentiment    0
dtype: int64

In [6]:
## helpers for data cleaning
punctuation = string.punctuation
stop_words = list(STOP_WORDS)
nlp = spacy.load("en_core_web_sm")
numbers = string.digits
# print(stop_words)
## function that cleans input text
def cleaning_function(input_text):
    text = nlp(input_text)
    tokens = []
    for token in text:
        temp = token.lemma_.lower()
        tokens.append(temp)

    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words and token not in punctuation and token not in numbers:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [7]:
## test
cleaned_text = cleaning_function(data["Review"][4])
print(cleaned_text)

['mic', 'great']


In [8]:
X = data["Review"]
y = data["Sentiment"]

In [9]:
## SVC using tfidf (bag of words)

tfidf = TfidfVectorizer(tokenizer = cleaning_function)
classifier = LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=50)
SVC_clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
SVC_clf.fit(X_train, y_train)
y_pred = SVC_clf.predict(X_test)

print("Classification Report for SVC:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix for SVC:")
print(confusion_matrix(y_test, y_pred))

Classification Report for SVC:
              precision    recall  f1-score   support

           0       0.75      0.82      0.79       154
           1       0.79      0.72      0.75       146

    accuracy                           0.77       300
   macro avg       0.77      0.77      0.77       300
weighted avg       0.77      0.77      0.77       300

Confusion Matrix for SVC:
[[126  28]
 [ 41 105]]


In [10]:
def predict_func(string):
    temp = SVC_clf.predict([string])
    if (temp[0]==0):
        return("Negative Review")
    else:
        return("Positive Review")

In [13]:
user_input_text = input("Enter review: ")
user_input_pred = predict_func(user_input_text)
print("-->",user_input_pred)

Enter review: Great value and build quality, keys are almost silent, one of the best typing experiences ever. RGB lights are pleasant, provided you like RGB at all. You can also switch them off, if you don't like them. Would buy it again and again. Strong recommendation along with HyperX Core keyboard.
--> Positive Review


In [14]:
## Add feature: input URL by user and output how many positive and negative comments

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get("http://www.python.org")

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
